Import Libraries

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical

Configs

In [5]:
DATASETS_TO_PROCESS = ['dataset1', 'dataset2']
PREPROCESSED_DIR = 'preprocessed_data'
RESULTS_DIR = 'results'
MODELS_DIR = 'models'
CNN_MODELS_DIR = os.path.join(MODELS_DIR, 'all_cnn_models') # save all cnn models to a sub directory
CM_DIR = os.path.join(RESULTS_DIR, 'confusion_matrices_cnn')
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(CNN_MODELS_DIR, exist_ok=True)
os.makedirs(CM_DIR, exist_ok=True)

final_summary_list = []

Helper functions

In [3]:
def plot_confusion_matrix_func(y_true, y_pred, class_names, title, output_path):
    cm = confusion_matrix(y_true, y_pred, labels=np.arange(len(class_names)))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label'); plt.ylabel('Actual Label')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f'saved confusion matrix to {output_path}')
    plt.show()

In [ ]:
for dataset_name in DATASETS_TO_PROCESS:
    print('\n' + '#' * 80)
    print(f'\nDATASET - {dataset_name.upper()}')
    print('\n' + '#' * 80)

    # load preprocessed data
    
    filepath = os.path.join(PREPROCESSED_DIR, f'{dataset_name}_processed.npz')
    with np.load(filepath, allow_pickle=True) as data:
        images, labels, class_map = data['images'], data['labels'], data['class_map'].item()
    
    class_names = list(class_map.keys())
    NUM_CLASSES = len(class_names)

    images_normalized = images / 255.0
    labels_categorical = to_categorical(labels, num_classes=NUM_CLASSES)

    X_train, X_test, y_train, y_test = train_test_split(
        images_normalized, labels_categorical, test_size=0.20, random_state=42, stratify=labels_categorical
    )
    
    # data augmentation layer

    data_augmentation = Sequential([
        RandomFlip("horizontal"), RandomRotation(0.1), RandomZoom(0.1),
    ], name='data_augmentation')

    # CNN architecture
    print('\n' + '=' * 60)
    print(f'\nbuilding CNN models and optimizing for {dataset_name}')
    print('\n' + '=' * 60)
    
    conv_blocks_options = [3, 4, 5]
    dropout_rate_options = [0.3, 0.5]
    learning_rate_options = [0.001, 0.0005]
    
    # hyperparameter search

    for conv_blocks in conv_blocks_options:
        for dropout_rate in dropout_rate_options:
            for learning_rate in learning_rate_options:
                
                param_str = f'blocks_{conv_blocks}_dropout_{dropout_rate}_LR_{learning_rate}'
                print(f'\n--- trying {param_str} ---')

                model = Sequential(name=param_str)
                model.add(tf.keras.Input(shape=X_train.shape[1:]))
                model.add(data_augmentation)

                for i in range(conv_blocks):
                    filters = 32 * (2**i)
                    model.add(Conv2D(filters, (3, 3), activation='relu', padding='same'))
                    model.add(BatchNormalization())
                    model.add(MaxPooling2D((2, 2)))
                
                model.add(Flatten())
                model.add(Dense(128, activation='relu'))
                model.add(Dropout(dropout_rate))
                model.add(Dense(NUM_CLASSES, activation='softmax'))

                model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
                
                early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
                reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

                model.fit(
                    X_train, y_train, epochs=50, batch_size=32,
                    validation_split=0.2, callbacks=[early_stopping, reduce_lr], verbose=0
                )
                
                _, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
                print(f'\naccuracy for {param_str}: {test_accuracy:.4f}')

                # save each combination

                model_path = os.path.join(CNN_MODELS_DIR, f'{dataset_name}_{param_str}.h5')
                model.save(model_path)
                
                y_pred = np.argmax(model.predict(X_test), axis=1)
                y_true = np.argmax(y_test, axis=1)
                
                # print confusion matrix for the combination
            
                cm_title = f'CM for {dataset_name}\n({param_str})'
                cm_output_path = os.path.join(CM_DIR, f'CM_{dataset_name}_{param_str}.png')
                plot_confusion_matrix_func(y_true, y_pred, class_names, cm_title, cm_output_path)
                
                report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

                final_summary_list.append({
                    'Dataset': dataset_name, 'Model': 'CNN', 'Conv Blocks': conv_blocks,
                    'Dropout Rate': dropout_rate, 'Learning Rate': learning_rate,
                    'Accuracy': test_accuracy, 'F1-Score (Macro)': report_dict['macro avg']['f1-score'],
                    'Precision (Macro)': report_dict['macro avg']['precision'], 'Recall (Macro)': report_dict['macro avg']['recall']
                })

Final CNN comparison table for the two datasets

In [ ]:
print('\n' + '#' * 80)
print('\n SUMMARY TABLE FOR ALL CNN MODELS')
print('\n' + '#' * 80)

df_comparison = pd.DataFrame(final_summary_list)
df_comparison = df_comparison.sort_values(by=['Dataset', 'Accuracy'], ascending=[True, False])
    
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1200)
    
print(df_comparison.to_string(index=False))
    
comparison_table_path = os.path.join(RESULTS_DIR, 'method2_cnn_full_summary.csv')
df_comparison.to_csv(comparison_table_path, index=False)
print(f'\nSaved CNN summary table to {comparison_table_path}')

print('\n\nDONE')